In [1]:
import gradio as gr
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys
from PIL import Image
import json
import os
import numpy as np
from util import *

face_detector = pipeline(Tasks.face_detection, model='gaosheng/face_detect')
# face_recognizer = pipeline(Tasks.face_recognition, model='damo/cv_ir101_facerecognition_cfglint')
face_recognizer = pipeline(Tasks.face_recognition, model='iic/cv_ir101_facerecognition_cfglint')
face_bank = load_face_bank('face_bank/', face_recognizer)

def inference(img: Image, draw_detect_enabled, detect_threshold, sim_threshold) -> json:
    img = resize_img(img)
    img = img.convert('RGB')
    detection_result = face_detector(img)

    boxes = np.array(detection_result[OutputKeys.BOXES])
    scores = np.array(detection_result[OutputKeys.SCORES])
    faces = []

    for i in range(len(boxes)):
        score = scores[i]
        if score < detect_threshold:
            continue
        box = boxes[i]
        face_embedding = get_face_embedding(img, box, face_recognizer)
        name, sim = get_name_sim(face_embedding, face_bank)
        if name is None:
            continue
        if sim < sim_threshold:
            faces.append({'box': box, 'name': '未知', 'sim': sim})
        else:
            faces.append({'box': box, 'name': name, 'sim': sim})
    rows = get_rows(faces)
    row_names = get_row_names(faces, rows)
    draw_name(img, row_names)
    if draw_detect_enabled:
        draw_faces(img, faces)
    return img, get_row_names_text(row_names)

examples = ['example.jpg']

with gr.Blocks() as demo:
    with gr.Row():
        draw_detect_enabled = gr.Checkbox(label="是否画框", value=False)
        detect_threshold = gr.Slider(label="检测阈值", minimum=0, maximum=1, value=0.5)
        sim_threshold = gr.Slider(label="识别阈值", minimum=0, maximum=1, value=0.3)
    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="pil", height=350)
            submit = gr.Button("提交")
        with gr.Column():
            img_output = gr.Image(type="pil")
            name_output = gr.Text(label="人名")
    submit.click(
        fn=inference, 
        inputs=[img_input, draw_detect_enabled, detect_threshold, sim_threshold],
        outputs=[img_output, name_output])
    gr.Examples(examples, inputs=[img_input])

demo.launch(share=True)

2024-12-08 07:54:32,397 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect
2024-12-08 07:54:32,398 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect.
2024-12-08 07:54:32,402 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect
/usr/local/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.21). Upgrade using: pip install -U albumen

load checkpoint from local path: /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect/pytorch_model.pt


2024-12-08 07:54:36,831 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2024-12-08 07:54:37,143 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint
2024-12-08 07:54:37,143 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint.
2024-12-08 07:54:37,156 - modelscope - WARNING - No preprocessor field found in cfg.
2024-12-08 07:54:37,156 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-12-08 07:54:37,156 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint'}. trying to build by task and model information.
2024-12-08 07:54:37,157 - modelscope - WARNING - Find task: face-recognition, model type: None. Insufficient information 

2024-12-08 07:54:37,944 - modelscope - WARNING - Model revision not specified, use revision: v1.1
2024-12-08 07:54:38,285 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd
2024-12-08 07:54:38,286 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd.
2024-12-08 07:54:38,290 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd
/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" inst

load checkpoint from local path: /mnt/workspace/.cache/modelscope/hub/damo/cv_ddsar_face-detection_iclr23-damofd/pytorch_model.pt


2024-12-08 07:54:39,703 - modelscope - INFO - face recognition model loaded!


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ad30ca5c15b7bd78b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
